In [ ]:

from multiprocessing import Process
from bson.objectid import ObjectId
from pymongo import MongoClient
from new_analyzer import run

class run_factor_integration:
    def __init__(self, keyid, fid):
        self.client =  MongoClient('203.255.92.141:27017', connect=False)
        self.PUBLIC = self.client['PUBLIC']
        self.new_max_factor = self.PUBLIC['new_factor'] 
        self.ID = self.client['ID']
        self.Domestic = self.ID['Domestic']
        self.keyid = keyid
        self.fid = fid
        
        self.DATA = self.ID['Domestic'].find({"keyId":self.keyid, "fid":fid})

    def count_people(self):
        cnt = 0
        for i in self.DATA:
            cnt += 1
        return cnt

    def run(self):
        cnt = self.count_people()
        processList = []
        if None == self.new_max_factor.find_one({'keyId': self.keyid}):
            self.new_max_factor.insert({'keyId': self.keyid},{'keyId': self.keyid, 'Quality' : -1, 'accuracy' : -1, 'recentness' : -1, 'coop': -1 })

        for i in range(0,cnt , 100):
            start = 1 *i
            end = 100
            if i//100 == cnt//100:
                start = i
                end = cnt
            
            proc = Process(target=run(start, end, self.fid, self.keyid),daemon = False)
            processList.append(proc)
            proc.start()

        for p in processList :
            p.join()
        
        self.factor_norm()

    def factor_norm(self):
        max_factor = self.new_max_factor.find({'keyId':self.keyid})
        for doc1 in max_factor:
            max_qual = doc1['Quality']
            update_list = self.Domestic.find({"keyId":self.keyid, 'fid': self.fid})
            for doc in update_list:
                if max_qual != 0:
                    norm_qual = doc['factor']['qual']/max_qual
                else:
                    norm_qual = doc['factor']['qual']
                score = norm_qual * 25 + doc['factor']['acc'] *25 + doc['factor']['recentness'] * 25 +  doc['factor']['coop'] * 25
                self.Domestic.update({'_id':ObjectId(doc['_id'])},{"$set":{'score':score ,'factor':{"qual":norm_qual,'coop':doc['factor']['coop'],'recentness':doc['factor']['recentness'],'acc':doc['factor']['acc']}}})
        print('end')
            
